# Spatially Informed Traveling Salesman Problem
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/17da8FuXsDfI9DCuaT7rbKKl2twoFYSlB#scrollTo=qO4f61z7Pc_A)
Developed by Wanhee Kim(Phd student at UTK) / Revised Date : 04/03/2024


## Sequence of spatially informed TSP
1. Measure adjacency based on spatial information
2. Generate distance matrix
3. Analyze TSP

## Analysis

In [ ]:
pip install spatialtsp

In [ ]:
# Make Toy Data
from spatialtsp import is_far_enough, generate_clustered_points
points=generate_clustered_points(10)
points

,geometry
0,POINT (7.00000 20.00000)
1,POINT (45.00000 39.00000)
2,POINT (11.00000 6.00000)
3,POINT (17.00000 12.00000)
4,POINT (20.00000 19.00000)
5,POINT (43.00000 35.00000)
6,POINT (38.00000 30.00000)
7,POINT (37.00000 33.00000)
8,POINT (8.00000 11.00000)
9,POINT (40.00000 30.00000)


In [ ]:
# Calculate Standard Distance Matrix
from spatialtsp import calculate_distance_matrix
distance_matrix=calculate_distance_matrix(points)
distance_matrix

array([[   0, 4249, 1456, 1281, 1304, 3900, 3257, 3270,  906, 3448],
       [4249,    0, 4738, 3890, 3202,  447, 1140, 1000, 4640, 1030],
       [1456, 4738,    0,  849, 1581, 4319, 3612, 3748,  583, 3764],
       [1281, 3890,  849,    0,  762, 3471, 2766, 2900,  906, 2921],
       [1304, 3202, 1581,  762,    0, 2802, 2110, 2202, 1442, 2283],
       [3900,  447, 4319, 3471, 2802,    0,  707,  632, 4244,  583],
       [3257, 1140, 3612, 2766, 2110,  707,    0,  316, 3551,  200],
       [3270, 1000, 3748, 2900, 2202,  632,  316,    0, 3640,  424],
       [ 906, 4640,  583,  906, 1442, 4244, 3551, 3640,    0, 3722],
       [3448, 1030, 3764, 2921, 2283,  583,  200,  424, 3722,    0]])

In [ ]:
# set working directory
# import os

# path = 'D:/GIS_analyzing/1.Standard_TSP/0.test_iteration' # write your own directory
# os.chdir(path)
# os.getcwd()

## Display Basemap

In [ ]:
## Add basemap
from spatialtsp import Map

my_map = Map(center=[40.7128, -74.0060], zoom=10)
my_map.add_basemap("OpenStreetMap.Mapnik")

my_map


Map(center=[40.7128, -74.006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [ ]:
## Add GeoJSON
from spatialtsp import Map

geojson_map = Map(center=[37.0902, -95.7129], zoom=4)
geojson_url = "https://github.com/opengeos/datasets/releases/download/us/us_states.geojson"
geojson_map.add_geojson(geojson_url, name="US Counties")

geojson_map


Map(center=[37.0902, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [ ]:
## Add shp
import geopandas as gpd
import requests
import zipfile
import io
from spatialtsp import Map

# Shapefile URL
url = 'https://github.com/opengeos/datasets/releases/download/us/us_states.zip'

# Download and extract the zip file
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))

# Find the shapefile name
shp_name = [name for name in z.namelist() if '.shp' in name][0]

# Read and extract the shapefile
z.extractall("temp_shp")
states_gdf = gpd.read_file("temp_shp/us_states.shp")


# Display the shapefile
my_map = Map(center=[37.0902, -95.7129], zoom=4)
my_map.add_vector(states_gdf, name="US States")

my_map


Map(center=[37.0902, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [ ]:
## Add Vector
from spatialtsp import Map

# Initialize the map
my_map = Map(center=[37.0902, -95.7129], zoom=4)

# GeoJSON URLs
city_geojson_url = "https://github.com/opengeos/datasets/releases/download/us/us_cities.geojson"
county_geojson_url = "https://github.com/opengeos/datasets/releases/download/us/us_counties.geojson"

# Shapefile URL
states_shape_url = 'https://github.com/opengeos/datasets/releases/download/us/us_states.zip'

# Load GeoJSONs as GeoDataFrames
gdf_cities = gpd.read_file(city_geojson_url)
gdf_counties = gpd.read_file(county_geojson_url)

# Use add_vector to add GeoDataFrames to the map
my_map.add_vector(gdf_counties, name="US Counties")
my_map.add_vector(gdf_cities, name="US Cities")
# For the shapefile, directly use the URL
my_map.add_vector(states_shape_url, name="US States")

# Display the map
my_map

Map(center=[37.0902, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [ ]:
## other codes will be updated soon